# Chapter 05. Model Deployment and Inference
---

## 5.1 Deploying a model for Batch Inference using SageMaker Batch Transform
---

In [1]:
# Change into the notebooks directory.
%cd /root/sagemaker-course/notebooks/

/root/sagemaker-course/notebooks


In [2]:
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

# S3 bucket information
BUCKET = 'sagemaker-course-20200812'
PREFIX = 'churn'
LOCAL_DATA_DIRECTORY = f'../data/{PREFIX}'
print(f"Artifacts will be written to s3://{BUCKET}/{PREFIX}")

# Session variables we'll use throughout the notebook
sagemaker_session = sagemaker.Session()
boto_session = sagemaker_session.boto_session
sagemaker_client = boto_session.client('sagemaker')
role = get_execution_role()
print(f'Role ARN: {role}')

Artifacts will be written to s3://sagemaker-course-20200812/churn
Role ARN: arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200618T144956


#### Step 1. Prepare Input Data


```
An example of input file content:
                Record1-Attribute1, Record1-Attribute2, Record1-Attribute3, ..., Record1-AttributeM
                Record2-Attribute1, Record2-Attribute2, Record2-Attribute3, ..., Record2-AttributeM
                Record3-Attribute1, Record3-Attribute2, Record3-Attribute3, ..., Record3-AttributeM
                ...
                RecordN-Attribute1, RecordN-Attribute2, RecordN-Attribute3, ..., RecordN-AttributeM
```         

In [3]:
!head -5 ../data/churn/test-batch.csv

186,0,137.8,97,187.7,118,146.4,85,8.7,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
112,17,183.2,95,252.8,125,156.7,95,9.7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
91,24,93.5,112,183.4,128,240.7,133,9.9,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
22,0,110.3,107,166.5,93,202.3,96,9.5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0


In [4]:
local_file_path = '../data/churn/test-batch.csv'

inference_data = sagemaker_session.upload_data(
    local_file_path,
    bucket=BUCKET,
    key_prefix=PREFIX)

inference_data

's3://sagemaker-course-20200812/churn/test-batch.csv'

#### Step 2. Initialize a `Tranformer` object

See the `sagemaker.model.Model` [API reference](https://sagemaker.readthedocs.io/en/stable/model.html) for more details.

In [5]:
from sagemaker import model

In [6]:
model_data = 's3://sagemaker-course-20200812/churn/builtin-xgboost-2020-08-22-16-28-08-204/output/model.tar.gz'
image_uri = '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

churn_model = model.Model(model_data=model_data,
                          image_uri=image_uri,
                          role=role)

See the `sagemaker.transformer.Transformer` [API reference](https://sagemaker.readthedocs.io/en/stable/transformer.html) for more details.

In [7]:
churn_transformer = churn_model.transformer(instance_count=1,
                                            instance_type='ml.m4.xlarge',
                                            strategy='MultiRecord',
                                            assemble_with='Line',
                                            output_path=f's3://{BUCKET}/{PREFIX}/transform')

### Step 3. Run Transform Job 

**NOTE**: This takes about 3-5 minutes to return.

In [8]:
# Start a transform job and wait for it to finish
churn_transformer.transform(data=inference_data,
                            content_type='text/csv',
                            split_type='Line')

.................................
.2020-09-05T14:28:33.631:[sagemaker logs]: MaxConcurrentTransforms=4, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
[2020-09-05:14:28:31:INFO] No GPUs detected (normal if no gpus installed)
[2020-09-05:14:28:31:INFO] No GPUs detected (normal if no gpus installed)
[2020-09-05:14:28:31:INFO] No GPUs detected (normal if no gpus installed)
[2020-09-05:14:28:31:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;

    keepalive_timeout 3;

    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
  

In [9]:
churn_transformer.output_path

's3://sagemaker-course-20200812/churn/transform'

In [10]:
# List files in S3 bucket
s3_client = boto_session.client('s3')
s3_client.list_objects(Bucket = BUCKET, Prefix = f'{PREFIX}/transform')

{'ResponseMetadata': {'RequestId': '8N4P4RDWEPAX1Y3G',
  'HostId': 'jVSVv2w/7v4wxXq6enhD/QWGL61l8O4AK+jCvxSiwl9WW8mOaNRL2HA8fPHz3RHV/0JSfswsRoU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jVSVv2w/7v4wxXq6enhD/QWGL61l8O4AK+jCvxSiwl9WW8mOaNRL2HA8fPHz3RHV/0JSfswsRoU=',
   'x-amz-request-id': '8N4P4RDWEPAX1Y3G',
   'date': 'Sat, 05 Sep 2020 14:33:18 GMT',
   'x-amz-bucket-region': 'us-east-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'churn/transform/test-batch.csv.out',
   'LastModified': datetime.datetime(2020, 9, 5, 14, 28, 34, tzinfo=tzlocal()),
   'ETag': '"c28b3d9b7a3050a23c05e8563368c40f"',
   'Size': 6793,
   'StorageClass': 'STANDARD'}],
 'Name': 'sagemaker-course-20200812',
 'Prefix': 'churn/transform',
 'MaxKeys': 1000,
 'EncodingType': 'url'}

In [11]:
# Download the output data from S3 to local filesystem
s3_client.download_file(
    Bucket=BUCKET,
    Key=f'{PREFIX}/transform/test-batch.csv.out',
    Filename=f'{LOCAL_DATA_DIRECTORY}/test-batch.csv.out')

In [12]:
!head -5 ../data/churn/test-batch.csv.out

0.010853796266019344
0.005068291909992695
0.008791499771177769
0.16663919389247894
0.004287515766918659


## 5.2 Creating a SageMaker Endpoint for Online Inference
---

See the `sagemaker.sklearn.SKLearnModel` [API reference](https://sagemaker.readthedocs.io/en/stable/sagemaker.sklearn.html#sagemaker.sklearn.model.SKLearnModel) for more details.

In [13]:
from sagemaker import sklearn

In [14]:
model_data = 's3://sagemaker-course-20200812/churn/custom-code-sklearn-2020-08-22-16-54-14-305/output/model.tar.gz'

sklearn_model = sklearn.SKLearnModel(framework_version='0.20.0',
                                     model_data=model_data, 
                                     role=role,
                                     entry_point='../scripts/sklearn/sklearn_rf.py')

See the `sagemaker.predictor.RealTimePredictor` [API reference](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-predictor) for more details.

**NOTE: This takes about 6-8 minutes to return.**

In [15]:
sklearn_predictor = sklearn_model.deploy(initial_instance_count=1,
                                         instance_type='ml.m4.xlarge')

-------------!

In [16]:
type(sklearn_predictor)

sagemaker.sklearn.model.SKLearnPredictor

In [17]:
df = pd.read_csv(f'{LOCAL_DATA_DIRECTORY}/test-dataset.csv', header=None)

# Remove first column which contains labels
X = df.drop(labels=0, axis=1)
X.head()

,1,2,3,4,5,6,7,8,9,10,...,60,61,62,63,64,65,66,67,68,69
0,186,0,137.8,97,187.7,118,146.4,85,8.7,6,...,0,0,0,0,0,1,1,0,1,0
1,132,25,113.2,96,269.9,107,229.1,87,7.1,7,...,0,0,0,0,1,0,1,0,0,1
2,112,17,183.2,95,252.8,125,156.7,95,9.7,3,...,0,0,0,0,1,0,1,0,0,1
3,91,24,93.5,112,183.4,128,240.7,133,9.9,3,...,0,0,0,0,0,1,0,1,0,1
4,22,0,110.3,107,166.5,93,202.3,96,9.5,5,...,0,0,0,1,0,0,1,0,1,0


In [18]:
preds = sklearn_predictor.predict(X)
print(preds)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0]


## 5.3 Configure Autoscaling for a Hosted Endpoint.

**To configure autoscaling for a model using the console**:

1. Open the Amazon SageMaker console at https://console.aws.amazon.com/sagemaker/.
2. In the navigation pane, choose **Endpoints**.
3. Choose the endpoint that you want to configure.
4. Under the **Endpoint runtime settings** heading, select the radio button corresponding to the model variant that you want to configure and click **Configure autoscaling**. The **Configure variant automatic scaling** page appears.
5. The **Variant automatic scaling** section lets us configure the min/max number of instances.
    * For **Minimum instance count**, type the minimum number of instances that you want the scaling policy to maintain. At least 1 instance is required.
    * For **Maximum instance count**, type the maximum number of instances that you want the scaling policy to maintain.
6. The **Built-in scaling policy** section lets us configure the conditions under which to scale the instances.
    * For the **Target value**, type the average number of invocations per instance per minute for the model. To determine this value, follow the guidelines in Load testing. Application Auto Scaling adds or removes instances to keep the metric close to the value that you specify.
    * For **Scale-in cool down** and **Scale-out cool down** type the number seconds for each cool down period. Assuming that the order in the list is based on either most important to less important of first applied to last applied.
    * Select **Disable scale in** to prevent the scaling policy from deleting variant instances if you want to ensure that your variant scales out to address increased traffic, but are not concerned with removing instances to reduce costs when traffic decreases. Scale-out activities are always enabled so that the scaling policy can create endpoint instances as needed.
7. Choose **Save**.

More details, including how to define a custom scaling policy, can be found in the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/endpoint-auto-scaling-add-console.html).

### Endpoint cleanup 

In [20]:
churn_model.delete_model()

In [21]:
sklearn_predictor.delete_endpoint()

In [22]:
sklearn_model.delete_model()

---
# Exercises


1. Create a `sagemaker.model.Model` object for the XGBoost model. Call the `prepare_container_def` method. What information is returned?
2. Deploy the xgboost Model object as an API endpoint. What is returned from the function call?
3. Use the deployed XGBoost model to predict on 1 sample of data.
4. Use the deployed XGBoost model to predict on multiple samples of data.
5. Create a `sagemaker.sklearn.SKLearnModel` object for the sklearn model. Call the `prepare_container_def` method. What information is returned? How does this object differ from what the XGBoost model returned when `prepare_container_def` was called?
6. Delete all `Models`, `Endpoint`, and `EndpointConfigs` that you created.

---
# Answers

### 1.

model_data = 's3://sagemaker-course-20200812/churn/builtin-xgboost-2020-08-22-16-28-08-204/output/model.tar.gz'
image_uri = '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

churn_model = model.Model(model_data=model_data,
                          image_uri=image_uri,
                          role=role)

In [20]:
churn_model.prepare_container_def()

{'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
 'Environment': {},
 'ModelDataUrl': 's3://sagemaker-course-20200812/churn/builtin-xgboost-2020-08-22-16-28-08-204/output/model.tar.gz'}

### 2.

In [21]:
cm = churn_model.deploy(endpoint_name='xgboost-churn-endpoint',
                        initial_instance_count=1,
                        instance_type='ml.m4.xlarge')

-------------!

In [22]:
type(cm)

NoneType

### 3.

In [23]:
from sagemaker import predictor

In [30]:
xgboost_predictor = predictor.Predictor(endpoint_name=churn_model.endpoint_name,
                                        sagemaker_session=sagemaker_session,
                                        serializer=sagemaker.serializers.CSVSerializer(),
                                        deserializer=sagemaker.deserializers.CSVDeserializer())

In [25]:
with open(f'{LOCAL_DATA_DIRECTORY}/test_sample.csv', 'r') as f:
    test_samples = f.readlines()

In [31]:
payload = test_samples[0].rstrip('\n')
xgboost_predictor.predict(data=payload)

[['0.014719205908477306']]

### 4.

In [32]:
num_samples = 5

multi_samples = [s.rstrip('\n') for s in test_samples[:num_samples]]
multi_samples

['186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55',
 '132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1',
 '112,17,183.2,95,252.8,125,156.7,95,9.7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1',
 '91,24,93.5,112,183.4,128,240.7,133,9.9,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1',
 '22,0,110.3,107,166.5,93,202.3,96,9.5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0']

In [33]:
xgboost_predictor.predict(data=multi_samples)

[['0.014719205908477306',
  '0.005068291909992695',
  '0.008791499771177769',
  '0.16663919389247894',
  '0.004287515766918659']]

### 5.

In [38]:
model_data = 's3://sagemaker-course-20200812/churn/custom-code-sklearn-2020-08-22-16-54-14-305/output/model.tar.gz'

sklearn_model = sklearn.SKLearnModel(framework_version='0.20.0',
                                     model_data=model_data, 
                                     role=role,
                                     entry_point='../scripts/sklearn/sklearn_rf.py',
                                     sagemaker_session=sagemaker_session)

In [39]:
sklearn_model.prepare_container_def()

{'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3',
 'Environment': {'SAGEMAKER_PROGRAM': 'sklearn_rf.py',
  'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-2-209970524256/sagemaker-scikit-learn-2020-09-05-15-05-55-744/sourcedir.tar.gz',
  'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
  'SAGEMAKER_REGION': 'us-east-2'},
 'ModelDataUrl': 's3://sagemaker-course-20200812/churn/custom-code-sklearn-2020-08-22-16-54-14-305/output/model.tar.gz'}

### 6.

In [ ]:
xgboost_predictor.delete_endpoint()

In [ ]:
churn_model.delete_model()